In [1]:
from typing import * 
from pathlib import Path

import pandas as pd
import numpy as np
import anndata as ad
import scanpy as sc
from tqdm import tqdm 
import matplotlib.pyplot as plt

from scrna_atlas import settings, utils

In [2]:
adata = ad.read_h5ad('/efs/expression_atlas/scrna/cellxgene/old/b84def55-a776-4aa4-a9a6-7aab8b973086.h5ad', backed='r')
# adata = ad.read_h5ad('/efs/expression_atlas/scrna/cellxgene/old/3310476e-ee9d-4179-9446-df5d073f38d8.h5ad', backed='r')

In [7]:
# Subset all of the tabula muris files. 

tabula_muris_files = []
tabula_sapiens_files = []
for fh in list(Path('/efs/expression_atlas/scrna/cellxgene/2023-12-15/h5ads/').glob('*.h5ad')):
    adata = ad.read_h5ad(fh, backed='r')
    print(adata.uns['title'])
    if 'A single-cell transcriptomic atlas characterizes ageing tissues in the mouse' in adata.uns['title']:
        tabula_muris_files.append(fh)
    if 'Tabula Sapiens' in adata.uns['title']:
        tabula_sapiens_files.append(fh)

Lung - A single-cell transcriptomic atlas characterizes ageing tissues in the mouse - 10x
Spatial transcriptomics in the healthy human kidney: Puck_200115_15
Individual Single-Cell RNA-seq PBMC Data from Lee et al.
Dissection: Thalamus (THM) - intralaminar nuclear complex (ILN) - posterior group of intralaminar nuclei (PILN) - centromedian and parafasicular nuclei - CM and Pf
Spatiotemporal analysis of human intestinal development at single-cell resolution: Fetal A6
An integrated transcriptomic and epigenomic atlas of mouse primary motor cortex cell types: 10X_nuclei_v3_Broad
UMAP visualization of fibroblast subclusters
Visium spatial - LV
Tabula Sapiens - Epithelial
Fovea - Cell Types of the Human Retina and Its Organoids at Single-Cell Resolution
Spatial transcriptomics in mouse: Puck_191223_17
white matter - oligodendroglia
DCM/ACM heart cell atlas: Macrophages
E9.5: 201112_05
Mature kidney dataset: full
Trophoblast_stemcells
Single cell RNA-seq data from normal adult kidney tissue


In [8]:
tabula_sapiens_files

[PosixPath('/efs/expression_atlas/scrna/cellxgene/2023-12-15/h5ads/97a17473-e2b1-4f31-a544-44a60773e2dd.h5ad'),
 PosixPath('/efs/expression_atlas/scrna/cellxgene/2023-12-15/h5ads/c5d88abe-f23a-45fa-a534-788985e93dad.h5ad'),
 PosixPath('/efs/expression_atlas/scrna/cellxgene/2023-12-15/h5ads/18eb630b-a754-4111-8cd4-c24ec80aa5ec.h5ad'),
 PosixPath('/efs/expression_atlas/scrna/cellxgene/2023-12-15/h5ads/e5c63d94-593c-4338-a489-e1048599e751.h5ad'),
 PosixPath('/efs/expression_atlas/scrna/cellxgene/2023-12-15/h5ads/cee11228-9f0b-4e57-afe2-cfe15ee56312.h5ad'),
 PosixPath('/efs/expression_atlas/scrna/cellxgene/2023-12-15/h5ads/ff45e623-7f5f-46e3-b47d-56be0341f66b.h5ad'),
 PosixPath('/efs/expression_atlas/scrna/cellxgene/2023-12-15/h5ads/e6a11140-2545-46bc-929e-da243eed2cae.h5ad'),
 PosixPath('/efs/expression_atlas/scrna/cellxgene/2023-12-15/h5ads/0d2ee4ac-05ee-40b2-afb6-ebb584caa867.h5ad'),
 PosixPath('/efs/expression_atlas/scrna/cellxgene/2023-12-15/h5ads/5a11f879-d1ef-458a-910c-9b0bdfca5ebf.

In [9]:
tabula_muris_files

[PosixPath('/efs/expression_atlas/scrna/cellxgene/2023-12-15/h5ads/0fb7916e-7a68-4a4c-a441-3ab3989f29a7.h5ad'),
 PosixPath('/efs/expression_atlas/scrna/cellxgene/2023-12-15/h5ads/b8c618e5-4b3d-4566-8a3f-7e40047f5c54.h5ad'),
 PosixPath('/efs/expression_atlas/scrna/cellxgene/2023-12-15/h5ads/3f4fe86f-aced-4d10-b174-ee35b9f46b9d.h5ad'),
 PosixPath('/efs/expression_atlas/scrna/cellxgene/2023-12-15/h5ads/4546e757-34d0-4d17-be06-538318925fcd.h5ad'),
 PosixPath('/efs/expression_atlas/scrna/cellxgene/2023-12-15/h5ads/76544818-bc5b-4a0d-87d4-40dde89545cb.h5ad'),
 PosixPath('/efs/expression_atlas/scrna/cellxgene/2023-12-15/h5ads/1d29fd10-c8b3-4611-b0ac-3c578125adbf.h5ad'),
 PosixPath('/efs/expression_atlas/scrna/cellxgene/2023-12-15/h5ads/98ad5247-68f8-42f8-b8e5-7938cb373a91.h5ad'),
 PosixPath('/efs/expression_atlas/scrna/cellxgene/2023-12-15/h5ads/f16a8f4d-bc97-43c5-a2f6-bbda952e4c5c.h5ad'),
 PosixPath('/efs/expression_atlas/scrna/cellxgene/2023-12-15/h5ads/821c79aa-044e-40cf-b331-0fe3edd48019.

In [3]:
cellxgene_loc = Path('/efs/expression_atlas/scrna/cellxgene/2023-12-15/h5ads/')

In [4]:
(settings.DATA_LOC / 'tabula_muris' / '10X').mkdir(exist_ok=True)
(settings.DATA_LOC / 'tabula_muris' / 'smart-seq').mkdir(exist_ok=True)

In [25]:
# for fh in tabula_muris_files:
for fh in (settings.DATA_LOC / 'tabula_muris').glob('*.h5ad'):
    print(fh.stem)
    _adata = ad.read_h5ad(cellxgene_loc / f'{fh.stem.split("_")[0]}.h5ad', backed='r')
    print(_adata.uns['title'])
    if '10x' in _adata.uns['title']:
        process_cellxgene_atlas_adatas(
            ad.read_h5ad(cellxgene_loc / f'{fh.stem.split("_")[0]}.h5ad', backed='r'),
            fh.stem.split('_')[0],
            out_loc=settings.DATA_LOC / 'tabula_muris' / '10X',
            use_rep='X_pca',
            umap_key='X_umap',
            feature_lengths_df=adata.var[['feature_length']].copy(),
        )
    elif 'Smart-seq2' in _adata.uns['title']:
        process_cellxgene_atlas_adatas(
            ad.read_h5ad(cellxgene_loc / f'{fh.stem.split("_")[0]}.h5ad', backed='r'),
            fh.stem.split('_')[0],
            out_loc=settings.DATA_LOC / 'tabula_muris' / 'smart-seq',
            use_rep='X_pca',
            umap_key='X_umap',
            feature_lengths_df=adata.var[['feature_length']].copy(),
            normalize_length=True,
        )
    elif _adata.obs['assay'].nunique() == 1 and '10X' in _adata.obs['assay'].values[0].upper():
        process_cellxgene_atlas_adatas(
            ad.read_h5ad(cellxgene_loc / f'{fh.stem.split("_")[0]}.h5ad', backed='r'),
            fh.stem.split('_')[0],
            out_loc=settings.DATA_LOC / 'tabula_muris' / '10X',
            use_rep='X_pca',
            umap_key='X_umap',
            feature_lengths_df=adata.var[['feature_length']].copy(),
        )
    elif _adata.obs['assay'].nunique() == 1 and 'SMART' in _adata.obs['assay'].values[0].upper():
        process_cellxgene_atlas_adatas(
            ad.read_h5ad(cellxgene_loc / f'{fh.stem.split("_")[0]}.h5ad', backed='r'),
            fh.stem.split('_')[0],
            out_loc=settings.DATA_LOC / 'tabula_muris' / 'smart-seq',
            use_rep='X_pca',
            umap_key='X_umap',
            feature_lengths_df=adata.var[['feature_length']].copy(),
            normalize_length=True,
        )
    else:
        print(f'Skipping {fh.stem}...')

0fb7916e-7a68-4a4c-a441-3ab3989f29a7_pseuodobulk
Lung - A single-cell transcriptomic atlas characterizes ageing tissues in the mouse - 10x
(17985, 9)
(17985, 10)


100%|██████████| 64/64 [00:58<00:00,  1.09it/s]
... storing 'tissue_cell_type' as categorical


3f4fe86f-aced-4d10-b174-ee35b9f46b9d_pseuodobulk
Trachea - A single-cell transcriptomic atlas characterizes ageing tissues in the mouse - Smart-seq2
(21069, 9)
(21069, 10)


100%|██████████| 34/34 [00:09<00:00,  3.58it/s]
... storing 'tissue_cell_type' as categorical


4546e757-34d0-4d17-be06-538318925fcd_pseuodobulk
Liver - A single-cell transcriptomic atlas characterizes ageing tissues in the mouse - Smart-seq2
(21069, 9)
(21069, 10)


...

98e5ea9f-16d6-47ec-a529-686e76515e39_pseuodobulk
All - A single-cell transcriptomic atlas characterizes ageing tissues in the mouse - Smart-seq2
(21069, 9)
(21069, 10)


100%|██████████| 532/532 [09:42<00:00,  1.10s/it]
... storing 'tissue_cell_type' as categorical


a6626b73-a0de-4dee-99aa-2559ab05af11_pseuodobulk
Gonadal fat pad — A single-cell transcriptomic atlas characterizes ageing tissues in the mouse
(21069, 9)
(21069, 10)


100%|██████████| 31/31 [00:06<00:00,  4.43it/s]
... storing 'tissue_cell_type' as categorical


f16a8f4d-bc97-43c5-a2f6-bbda952e4c5c_pseuodobulk
All - A single-cell transcriptomic atlas characterizes ageing tissues in the mouse
Skipping f16a8f4d-bc97-43c5-a2f6-bbda952e4c5c_pseuodobulk...


In [5]:
(settings.DATA_LOC / 'tabula_sapiens' / '10X').mkdir(exist_ok=True)
(settings.DATA_LOC / 'tabula_sapiens' / 'smart-seq').mkdir(exist_ok=True)

In [7]:
# for fh in tabula_sapiens_files:
for fh in (settings.DATA_LOC / 'tabula_sapiens').glob('*.h5ad'):
    print(fh.stem)
    _adata = ad.read_h5ad(cellxgene_loc / f'{fh.stem.split("_")[0]}.h5ad')
    _adata_10X = _adata[_adata.obs['assay'].str.upper().str.contains('10X')]
    _adata_smartseq = _adata[_adata.obs['assay'].str.upper().str.contains('SMART')]
    if _adata_10X.shape[0] > 0:
        process_cellxgene_atlas_adatas(
            # ad.read_h5ad(cellxgene_loc / f'{fh.stem.split("_")[0]}.h5ad', backed='r'),
            _adata_10X,
            fh.stem.split('_')[0],
            out_loc=settings.DATA_LOC / 'tabula_sapiens' / '10X',
            use_rep='X_scvi',
            # umap_key='X_scvi_umap',
            umap_key='X_umap',
            feature_lengths_df=adata.var[['feature_length']].copy(),
            normalize_length=False,
        )
    if _adata_smartseq.shape[0] > 0:
        process_cellxgene_atlas_adatas(
            # ad.read_h5ad(cellxgene_loc / f'{fh.stem.split("_")[0]}.h5ad', backed='r'),
            _adata_smartseq,
            fh.stem.split('_')[0],
            out_loc=settings.DATA_LOC / 'tabula_sapiens' / 'smart-seq',
            use_rep='X_scvi',
            # umap_key='X_scvi_umap',
            umap_key='X_umap',
            feature_lengths_df=adata.var[['feature_length']].copy(),
            normalize_length=True,
        )


18eb630b-a754-4111-8cd4-c24ec80aa5ec_pseuodobulk


/home/ec2-user/anaconda/envs/singlecell/lib/python3.9/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/home/ec2-user/anaconda/envs/singlecell/lib/python3.9/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/home/ec2-user/anaconda/envs/singlecell/lib/python3.9/site-pac

(58604, 12)
(58604, 13)


100%|██████████| 136/136 [00:44<00:00,  3.08it/s]
... storing 'tissue_cell_type' as categorical


(58604, 12)
(58604, 13)


...

100%|██████████| 33/33 [00:00<00:00, 34.84it/s]
... storing 'tissue_cell_type' as categorical


7357cee7-9f7f-4ab0-8cec-90de8f047e38_pseuodobulk
(58604, 12)
(58604, 13)


100%|██████████| 44/44 [00:11<00:00,  3.94it/s]
... storing 'tissue_cell_type' as categorical


(58604, 12)
(58604, 13)


100%|██████████| 15/15 [00:00<00:00, 73.51it/s]
... storing 'tissue_cell_type' as categorical


In [5]:
atlas_adata_muris = ad.read_h5ad(cellxgene_loc / '48b37086-25f7-4ecd-be66-f5bb378e3aea.h5ad', backed='r')

In [6]:
padata = ad.read_h5ad(settings.DATA_LOC / 'tabula_muris' / '10X' / '48b37086-25f7-4ecd-be66-f5bb378e3aea_pseuodobulk.h5ad')

In [8]:
padata_muris_tissue = utils.pseudobulk_adata(
    atlas_adata_muris,
    'tissue',
    'tissue',
    min_cells=5,
    min_counts=200,
    umap_key='X_umap',
    calc_tpm=True,
    normalize_length=False,
)

100%|██████████| 16/16 [03:29<00:00, 13.11s/it]


In [9]:
padata_muris_celltype = utils.pseudobulk_adata(
    atlas_adata_muris,
    'cell_type',
    'cell_type',
    min_cells=5,
    min_counts=200,
    umap_key='X_umap',
    calc_tpm=True,
    normalize_length=False,
)

100%|██████████| 119/119 [10:26<00:00,  5.26s/it]


In [10]:
padata_muris_louvain = utils.pseudobulk_adata(
    atlas_adata_muris,
    'louvain',
    'louvain',
    min_cells=5,
    min_counts=200,
    umap_key='X_umap',
    calc_tpm=True,
    normalize_length=False,
)

100%|██████████| 58/58 [11:04<00:00, 11.45s/it]


In [11]:
atlas_adata_muris.obs['tissue_cell_type'] = atlas_adata_muris.obs.apply(lambda x: f'{x.tissue}__{x.cell_type}', axis=1)

In [12]:
padata_muris_tissuecelltype = utils.pseudobulk_adata(
    atlas_adata_muris,
    'tissue_cell_type',
    'tissue_cell_type',
    min_cells=5,
    min_counts=200,
    umap_key='X_umap',
    calc_tpm=True,
    normalize_length=False,
)

100%|██████████| 166/166 [10:24<00:00,  3.76s/it]


In [13]:
padata.var['tau__tissue'] = calc_adata_tau(padata_muris_tissue, return_tau=True)
padata.var['tau__cell_type'] = calc_adata_tau(padata_muris_celltype, return_tau=True)
padata.var['tau__louvain'] = calc_adata_tau(padata_muris_louvain, return_tau=True)
padata.var['tau__tissue_cell_type'] = calc_adata_tau(padata_muris_tissuecelltype, return_tau=True)
padata.var['tau__pseudobulk'] = calc_adata_tau(padata, return_tau=True)

/tmp/ipykernel_4609/1313015571.py:14: RuntimeWarning: invalid value encountered in divide
  xi = l / l.max(axis=0)
/tmp/ipykernel_4609/1313015571.py:14: RuntimeWarning: invalid value encountered in divide
  xi = l / l.max(axis=0)
/tmp/ipykernel_4609/1313015571.py:14: RuntimeWarning: invalid value encountered in divide
  xi = l / l.max(axis=0)
/tmp/ipykernel_4609/1313015571.py:14: RuntimeWarning: invalid value encountered in divide
  xi = l / l.max(axis=0)
/tmp/ipykernel_4609/1313015571.py:14: RuntimeWarning: invalid value encountered in divide
  xi = l / l.max(axis=0)


In [19]:
s_tissue = utils.label_expression_similarity(
    padata_muris_tissue.layers['ntpm'],
    padata_muris_tissue.obs,
    padata_muris_tissue.var,
    specificity_column = 'tissue',
)
s_celltype = utils.label_expression_similarity(
    padata_muris_celltype.layers['ntpm'],
    padata_muris_celltype.obs,
    padata_muris_celltype.var,
    specificity_column = 'cell_type',
)
s_tissuecelltype = utils.label_expression_similarity(
    padata_muris_tissuecelltype.layers['ntpm'],
    padata_muris_tissuecelltype.obs,
    padata_muris_tissuecelltype.var,
    specificity_column = 'tissue_cell_type',
)
s_louvain = utils.label_expression_similarity(
    padata_muris_louvain.layers['ntpm'],
    padata_muris_louvain.obs,
    padata_muris_louvain.var,
    specificity_column = 'louvain',
)
s_pseudobulk = utils.label_expression_similarity(
    padata.layers['ntpm'],
    padata.obs,
    padata.var,
)
padata.uns['specificity'] = {}
padata.uns['specificity']['tissue'] = s_tissue
padata.uns['specificity']['cell_type'] = s_celltype
padata.uns['specificity']['tissue_cell_type'] = s_tissuecelltype
padata.uns['specificity']['louvain'] = s_louvain
padata.uns['specificity']['pseudobulk'] = s_pseudobulk

In [22]:
(settings.DATA_LOC / 'tabula_muris' / '10X' / 'tmp').mkdir(exist_ok=True)
padata.write_h5ad(settings.DATA_LOC / 'tabula_muris' / '10X' / 'tmp' / '48b37086-25f7-4ecd-be66-f5bb378e3aea_pseuodobulk.h5ad')

In [8]:
atlas_adata_sapiens = ad.read_h5ad(cellxgene_loc / '53d208b0-2cfd-4366-9866-c3c6114081bc.h5ad')

In [9]:
padata = ad.read_h5ad(settings.DATA_LOC / 'tabula_sapiens' / '10X' / '53d208b0-2cfd-4366-9866-c3c6114081bc_pseuodobulk.h5ad')

In [10]:
padata_sapiens_tissue = utils.pseudobulk_adata(
    atlas_adata_sapiens,
    'tissue',
    'tissue',
    min_cells=5,
    min_counts=200,
    umap_key='X_umap',
    calc_tpm=True,
    normalize_length=False,
)

  2%|▏         | 1/45 [00:11<08:32, 11.65s/it]2024-06-25 04:26:43.450 WARNING root: Singular matrix. Likely caused by all points lying in an N-1 space.
2024-06-25 04:26:43.525 WARNING root: Singular matrix. Likely caused by all points lying in an N-1 space.
2024-06-25 04:26:43.611 WARNING root: Singular matrix. Likely caused by all points lying in an N-1 space.
2024-06-25 04:26:43.614 WARNING root: Singular matrix. Likely caused by all points lying in an N-1 space.
2024-06-25 04:26:43.615 WARNING root: Singular matrix. Likely caused by all points lying in an N-1 space.
2024-06-25 04:26:43.620 WARNING root: Singular matrix. Likely caused by all points lying in an N-1 space.
  7%|▋         | 3/45 [00:24<05:29,  7.84s/it]2024-06-25 04:27:20.192 WARNING root: Singular matrix. Likely caused by all points lying in an N-1 space.
2024-06-25 04:27:20.451 WARNING root: Singular matrix. Likely caused by all points lying in an N-1 space.
2024-06-25 04:27:22.381 WARNING root: Singular matrix. Likel

In [11]:
padata_sapiens_celltype = utils.pseudobulk_adata(
    atlas_adata_sapiens,
    'cell_type',
    'cell_type',
    min_cells=5,
    min_counts=200,
    umap_key='X_umap',
    calc_tpm=True,
    normalize_length=False,
)

 11%|█▏        | 18/160 [01:20<06:46,  2.86s/it]2024-06-25 04:36:07.421 WARNING root: Singular matrix. Likely caused by all points lying in an N-1 space.
2024-06-25 04:36:08.250 WARNING root: Singular matrix. Likely caused by all points lying in an N-1 space.
2024-06-25 04:36:08.336 WARNING root: Singular matrix. Likely caused by all points lying in an N-1 space.
2024-06-25 04:36:08.386 WARNING root: Singular matrix. Likely caused by all points lying in an N-1 space.
2024-06-25 04:36:08.597 WARNING root: Singular matrix. Likely caused by all points lying in an N-1 space.
 13%|█▎        | 21/160 [01:34<08:00,  3.46s/it]2024-06-25 04:36:24.765 WARNING root: Singular matrix. Likely caused by all points lying in an N-1 space.
2024-06-25 04:36:25.044 WARNING root: Singular matrix. Likely caused by all points lying in an N-1 space.
2024-06-25 04:36:25.299 WARNING root: Singular matrix. Likely caused by all points lying in an N-1 space.
2024-06-25 04:36:25.634 WARNING root: Singular matrix. L

In [18]:
atlas_adata_sapiens.obs

,tissue_in_publication,assay_ontology_term_id,donor_id,anatomical_information,n_counts_UMIs,n_genes,cell_ontology_class,free_annotation,manually_annotated,compartment,...,cell_type,assay,disease,organism,sex,tissue,self_reported_ethnicity,development_stage,_sample_key,_cell_index
AAACCCACACTCCTGT_TSP6_Liver_NA_10X_1_1,Liver,EFO:0009922,TSP6,nan,7633.0,2259,macrophage,Monocyte/Macrophage,True,immune,...,macrophage,10x 3' v3,normal,Homo sapiens,male,liver,European,67-year-old human stage,macrophage__macrophage,0
AAACGAAGTACCAGAG_TSP6_Liver_NA_10X_1_1,Liver,EFO:0009922,TSP6,nan,2858.0,1152,monocyte,Monocyte,True,immune,...,monocyte,10x 3' v3,normal,Homo sapiens,male,liver,European,67-year-old human stage,monocyte__monocyte,1
AAACGCTCAACGGCTC_TSP6_Liver_NA_10X_1_1,Liver,EFO:0009922,TSP6,nan,7787.0,2983,endothelial cell of hepatic sinusoid,Endothelial,True,endothelial,...,endothelial cell of hepatic sinusoid,10x 3' v3,normal,Homo sapiens,male,liver,European,67-year-old human stage,endothelial cell of hepatic sinusoid__endothel...,2
AAAGAACAGCCTCTTC_TSP6_Liver_NA_10X_1_1,Liver,EFO:0009922,TSP6,nan,10395.0,2598,macrophage,Monocyte/Macrophage,True,immune,...,macrophage,10x 3' v3,normal,Homo sapiens,male,liver,European,67-year-old human stage,macrophage__macrophage,3
AAAGAACGTAGCACAG_TSP6_Liver_NA_10X_1_1,Liver,EFO:0009922,TSP6,nan,6610.0,2125,liver dendritic cell,Dendritic cell,True,immune,...,liver dendritic cell,10x 3' v3,normal,Homo sapiens,male,liver,European,67-year-old human stage,liver dendritic cell__liver dendritic cell,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TSP2_Vasculature_aorta_SS2_B114577_B133059_Endothelial_P4_S364,Vasculature,EFO:0008931,TSP2,aorta,13205.0,579,endothelial cell,endothelial cell,True,endothelial,...,endothelial cell,Smart-seq2,normal,Homo sapiens,female,aorta,African American or Afro-Caribbean,61-year-old human stage,endothelial cell__endothelial cell,483147
TSP2_Vasculature_aorta_SS2_B114577_B133059_Endothelial_P5_S365,Vasculature,EFO:0008931,TSP2,aorta,9565.0,529,endothelial cell,endothelial cell,True,endothelial,...,endothelial cell,Smart-seq2,normal,Homo sapiens,female,aorta,African American or Afro-Caribbean,61-year-old human stage,endothelial cell__endothelial cell,483148
TSP2_Vasculature_aorta_SS2_B114577_B133059_Endothelial_P7_S367,Vasculature,EFO:0008931,TSP2,aorta,195639.0,2753,endothelial cell,endothelial cell,True,endothelial,...,endothelial cell,Smart-seq2,normal,Homo sapiens,female,aorta,African American or Afro-Caribbean,61-year-old human stage,endothelial cell__endothelial cell,483149
TSP2_Vasculature_aorta_SS2_B114577_B133059_Endothelial_P8_S368,Vasculature,EFO:0008931,TSP2,aorta,37260.0,984,endothelial cell,endothelial cell,True,endothelial,...,endothelial cell,Smart-seq2,normal,Homo sapiens,female,aorta,African American or Afro-Caribbean,61-year-old human stage,endothelial cell__endothelial cell,483150


In [19]:
sc.pp.neighbors(
    atlas_adata_sapiens,
    n_neighbors=15,
    n_pcs=50,
    use_rep='X_scvi',
)

sc.tl.louvain(
    atlas_adata_sapiens,
    resolution=1.0,
)

In [20]:
padata_sapiens_louvain = utils.pseudobulk_adata(
    atlas_adata_sapiens,
    'louvain',
    'louvain',
    min_cells=5,
    min_counts=200,
    umap_key='X_umap',
    calc_tpm=True,
    normalize_length=False,
)

  0%|          | 0/78 [00:00<?, ?it/s]2024-06-25 04:55:58.428 WARNING root: Singular matrix. Likely caused by all points lying in an N-1 space.
2024-06-25 04:55:58.678 WARNING root: Singular matrix. Likely caused by all points lying in an N-1 space.
  3%|▎         | 2/78 [00:39<22:45, 17.96s/it]2024-06-25 04:56:24.322 WARNING root: Singular matrix. Likely caused by all points lying in an N-1 space.
2024-06-25 04:56:24.638 WARNING root: Singular matrix. Likely caused by all points lying in an N-1 space.
2024-06-25 04:56:24.638 WARNING root: Singular matrix. Likely caused by all points lying in an N-1 space.
2024-06-25 04:56:24.941 WARNING root: Singular matrix. Likely caused by all points lying in an N-1 space.
2024-06-25 04:56:25.093 WARNING root: Singular matrix. Likely caused by all points lying in an N-1 space.
 12%|█▏        | 9/78 [01:50<11:52, 10.33s/it]2024-06-25 04:57:32.982 WARNING root: Singular matrix. Likely caused by all points lying in an N-1 space.
2024-06-25 04:57:33.37

In [21]:
atlas_adata_sapiens.obs['tissue_cell_type'] = atlas_adata_sapiens.obs.apply(lambda x: f'{x.tissue}__{x.cell_type}', axis=1)

In [22]:
padata_sapiens_tissuecelltype = utils.pseudobulk_adata(
    atlas_adata_sapiens,
    'tissue_cell_type',
    'tissue_cell_type',
    min_cells=5,
    min_counts=200,
    umap_key='X_umap',
    calc_tpm=True,
    normalize_length=False,
)

  2%|▏         | 16/679 [00:10<03:11,  3.46it/s]2024-06-25 05:03:40.510 WARNING root: Singular matrix. Likely caused by all points lying in an N-1 space.
2024-06-25 05:03:40.598 WARNING root: Singular matrix. Likely caused by all points lying in an N-1 space.
2024-06-25 05:03:40.619 WARNING root: Singular matrix. Likely caused by all points lying in an N-1 space.
2024-06-25 05:03:40.729 WARNING root: Singular matrix. Likely caused by all points lying in an N-1 space.
2024-06-25 05:03:40.763 WARNING root: Singular matrix. Likely caused by all points lying in an N-1 space.
  6%|▌         | 38/679 [00:26<11:00,  1.03s/it]2024-06-25 05:03:57.406 WARNING root: Singular matrix. Likely caused by all points lying in an N-1 space.
2024-06-25 05:03:57.839 WARNING root: Singular matrix. Likely caused by all points lying in an N-1 space.
 15%|█▍        | 99/679 [01:54<05:08,  1.88it/s]2024-06-25 05:05:24.209 WARNING root: Singular matrix. Likely caused by all points lying in an N-1 space.
2024-06-

In [28]:
padata.var['tau__tissue'] = calc_adata_tau(padata_sapiens_tissue, return_tau=True)
padata.var['tau__cell_type'] = calc_adata_tau(padata_sapiens_celltype, return_tau=True)
padata.var['tau__louvain'] = calc_adata_tau(padata_sapiens_louvain, return_tau=True)
padata.var['tau__tissue_cell_type'] = calc_adata_tau(padata_sapiens_tissuecelltype, return_tau=True)
padata.var['tau__pseudobulk'] = calc_adata_tau(padata, return_tau=True)

/tmp/ipykernel_23071/1313015571.py:14: RuntimeWarning: invalid value encountered in divide
  xi = l / l.max(axis=0)
/tmp/ipykernel_23071/1313015571.py:14: RuntimeWarning: invalid value encountered in divide
  xi = l / l.max(axis=0)
/tmp/ipykernel_23071/1313015571.py:14: RuntimeWarning: invalid value encountered in divide
  xi = l / l.max(axis=0)
/tmp/ipykernel_23071/1313015571.py:14: RuntimeWarning: invalid value encountered in divide
  xi = l / l.max(axis=0)
/tmp/ipykernel_23071/1313015571.py:14: RuntimeWarning: invalid value encountered in divide
  xi = l / l.max(axis=0)


In [29]:
s_tissue = utils.label_expression_similarity(
    padata_sapiens_tissue.layers['ntpm'],
    padata_sapiens_tissue.obs,
    padata_sapiens_tissue.var,
    specificity_column = 'tissue',
)
s_celltype = utils.label_expression_similarity(
    padata_sapiens_celltype.layers['ntpm'],
    padata_sapiens_celltype.obs,
    padata_sapiens_celltype.var,
    specificity_column = 'cell_type',
)
s_tissuecelltype = utils.label_expression_similarity(
    padata_sapiens_tissuecelltype.layers['ntpm'],
    padata_sapiens_tissuecelltype.obs,
    padata_sapiens_tissuecelltype.var,
    specificity_column = 'tissue_cell_type',
)
s_louvain = utils.label_expression_similarity(
    padata_sapiens_louvain.layers['ntpm'],
    padata_sapiens_louvain.obs,
    padata_sapiens_louvain.var,
    specificity_column = 'louvain',
)
s_pseudobulk = utils.label_expression_similarity(
    padata.layers['ntpm'],
    padata.obs,
    padata.var,
)
padata.uns['specificity'] = {}
padata.uns['specificity']['tissue'] = s_tissue
padata.uns['specificity']['cell_type'] = s_celltype
padata.uns['specificity']['tissue_cell_type'] = s_tissuecelltype
padata.uns['specificity']['louvain'] = s_louvain
padata.uns['specificity']['pseudobulk'] = s_pseudobulk